In [1]:
%matplotlib inline
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
import lightgbm as lgb

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 3000)

In [41]:
#train_df = pd.read_csv('C:/D_Disk/data_competition/home_credit/all/application_train.csv', index_col=0, header=0)
#test_df = pd.read_csv('C:/D_Disk/data_competition/home_credit/all/application_test.csv', index_col=0, header=0)

#X_train_new, X_val_new, y_train_new, y_val_new = train_test_split(merged_X_train, train_y, test_size=0.2, random_state=42)
#print(train_df.columns)

train_df = pd.read_csv('C:/D_Disk/data_competition/home_credit/all/application_train.csv', header=0)
test_df = pd.read_csv('C:/D_Disk/data_competition/home_credit/all/application_test.csv', header=0)
train_y = train_df['TARGET']
train_X = train_df.drop(['TARGET'], axis=1)
test_X = test_df
merged_X = train_X.append(test_X)
merged_X_train = merged_X[:len(train_y)]
merged_X_test = merged_X[len(train_y):]

print('len of train_y: {} merged_X.shape: {} merged_X_train.shape: {} train_y.shape: {} '.format(
       len(train_y), merged_X.shape, merged_X_train.shape, train_y.shape))


len of train_y: 307511 merged_X.shape: (356255, 121) merged_X_train.shape: (307511, 121) train_y.shape: (307511,) 


In [34]:
bureau_df = pd.read_csv('C:/D_Disk/data_competition/home_credit/all/bureau.csv', header=0)
bureau_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [51]:
bureau_df['DAYS_CREDIT_ENDDATE'].value_counts()

 0.0        883
 3.0        845
-7.0        837
 1.0        830
-14.0       787
-10.0       782
 4.0        777
-2.0        772
-1.0        771
-42.0       768
-16.0       765
-6.0        760
-11.0       750
-20.0       746
-21.0       745
-13.0       744
-8.0        742
-22.0       740
-29.0       740
-12.0       740
-119.0      738
 6.0        735
-15.0       734
 7.0        734
-36.0       727
 5.0        726
-43.0       723
-63.0       716
-35.0       710
-26.0       708
-18.0       707
-28.0       707
-25.0       706
-31.0       705
-3.0        705
-4.0        704
 2.0        704
-34.0       703
-64.0       701
-53.0       700
-112.0      699
-168.0      699
-17.0       696
-5.0        695
-23.0       693
-329.0      692
-77.0       689
-47.0       688
-40.0       686
-24.0       686
-154.0      682
-84.0       682
-49.0       680
-48.0       679
-33.0       678
-78.0       677
-91.0       677
-238.0      676
-9.0        675
-127.0      674
-27.0       674
 10.0       674
-57.0   

In [30]:
bureau_balance_df = pd.read_csv('C:/D_Disk/data_competition/home_credit/all/bureau_balance.csv', header=0)
bureau_balance_df.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [46]:
bureau_joined_df = pd.merge(bureau_df, bureau_balance_df, on='SK_ID_BUREAU')
bureau_df_by_skid = bureau_joined_df.groupby('SK_ID_CURR')

bureau_joined_df.head(30)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE,STATUS
0,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,0,C
1,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,-1,C
2,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,-2,C
3,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,-3,C
4,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,-4,C
5,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,-5,C
6,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,-6,C
7,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,-7,C
8,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,-8,C
9,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,-9,0


In [36]:
bureau_joined_df.shape, bureau_df.shape, bureau_balance_df.shape

((24179741, 19), (1716428, 17), (27299925, 3))

In [40]:
bureau_joined_df[bureau_joined_df['SK_ID_CURR'].isna()].head()
bureau_joined_df['AMT_CREDIT_SUM_OVERDUE'].isna().sum()

0

In [42]:
POS_CASH_balance_df = pd.read_csv('C:/D_Disk/data_competition/home_credit/all/POS_CASH_balance.csv', header=0)

In [45]:
POS_CASH_balance_df[POS_CASH_balance_df['SK_ID_PREV']==1624618]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
6375005,1624618,362520,-80,4.0,0.0,Active,1493,0
6378612,1624618,362520,-38,4.0,0.0,Active,2770,0
7840228,1624618,362520,-42,4.0,0.0,Active,2649,0
7840249,1624618,362520,-44,4.0,0.0,Active,2588,0
7840486,1624618,362520,-34,4.0,0.0,Active,2892,0
7840639,1624618,362520,-24,4.0,0.0,Active,3196,0
7840655,1624618,362520,-53,4.0,0.0,Active,2314,0
7840811,1624618,362520,-40,4.0,0.0,Active,2710,0
7840915,1624618,362520,-46,4.0,0.0,Active,2526,0
7841029,1624618,362520,-31,4.0,0.0,Active,2984,0


In [14]:
# df2.groupby(['item_brand_id'])['item_id'].count()
df2['item_brand_id'].count()

18371

In [10]:
7547755734838966011 in sss2

True